In [ ]:
!pip install --quiet pandas numpy scanpy statsmodels scipy matplotlib seaborn umap-learn loompy python-igraph

     |████████████████████████████████| 2.0 MB 5.3 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 41 kB 90 kB/s 
     |████████████████████████████████| 91 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 62.5 MB/s 
     |████████████████████████████████| 48 kB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 52.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


In [ ]:
!pip install --quiet git+https://github.com/pcahan1/PySingleCellNet/

In [ ]:
def scn_predict(cgenes, xpairs, rf_tsp, aDat, nrand = 2):
###    expDat= pd.DataFrame(data=aDat.X, index= aDat.obs.index.values, columns= aDat.var.index.values)
    expDat= pd.DataFrame(data=aDat.X.toarray(), index= aDat.obs.index.values, columns= aDat.var.index.values)
    expValTrans=query_transform(expDat.reindex(labels=cgenes, axis='columns', fill_value=0), xpairs)
    classRes_val=rf_classPredict(rf_tsp, expValTrans, numRand=nrand)
    return classRes_val

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
import scanpy as sc
import scipy as sp
import numpy as np
import anndata
import loompy
import pickle
from scipy import stats
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as pl
from matplotlib import rcParams
from sklearn import svm

sc.settings.verbosity = 3 
sc.logging.print_header()

import pySingleCellNet as pySCN

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


scanpy==1.8.2 anndata==0.7.8 umap==0.5.2 numpy==1.19.5 scipy==1.4.1 pandas==1.1.5 scikit-learn==1.0.1 statsmodels==0.10.2 python-igraph==0.9.9 pynndescent==0.5.5


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load dataset first

In [ ]:
adTrain = sc.read_h5ad("/content/drive/MyDrive/KPMP/snRNA/integration/human4integration_01042022.h5ad")
adQuery = sc.read_h5ad("/content/drive/MyDrive/KPMP/snRNA/integration/mouse4integration_01042022.h5ad")

In [ ]:
train=sc.AnnData(X=adTrain.X,obs=adTrain.obs,var=adTrain.var)
query=sc.AnnData(X=adQuery.X,obs=adQuery.obs,var=adQuery.var)

In [ ]:
train.var.index=train.var['features']
query.var.index=query.var['features']

# Annotation 1: PT.1, PT.2 as healthy, PT.3 as injury, PT.4 as FR

In [ ]:
celltypedict={"Healthy_S1S2":"PT_Healthy","Healthy_S3":"PT_Healthy","Fib":"Fib","EC1":"EC","Failed_Repair_PT":"PT_Injury","DCT":"DCT","CTAL1":"TAL","CTAL2":"TAL",
              "DTL-ATL":"DTL-ATL","ICB":"ICB","CNT":"CNT","DCT-CNT":"DCT","ICA":"ICA","MTAL":"TAL","Mo":"Myeloid","PC1":"PC","Per":"Pericyte","PC2":"PC","Tcell":"Lymphocyte",
              "POD":"POD","PEC":"PEC","URO":"URO","MD":"MD","EC2":"EC","Repairing_PT":"PT_Injury","Injured_S1S2":"PT_Injury","Severe_Injured_PT":"PT_Injury",
              "Injured_S3":"PT_Injury"}
query.obs['celltype_merged']=query.obs['celltype2'].map(celltypedict)

In [ ]:
celltypedict={"Healthy_S1S2":"PT","Healthy_S3":"PT","Fib":"Fib","EC1":"EC","Failed_Repair_PT":"PT","DCT":"DCT","CTAL1":"TAL","CTAL2":"TAL",
              "DTL-ATL":"DTL-ATL","ICB":"ICB","CNT":"CNT","DCT-CNT":"DCT","ICA":"ICA","MTAL":"TAL","Mo":"Myeloid","PC1":"PC","Per":"Pericyte","PC2":"PC","Tcell":"Lymphocyte",
              "POD":"POD","PEC":"PEC","URO":"URO","MD":"MD","EC2":"EC","Repairing_PT":"PT","Injured_S1S2":"PT","Severe_Injured_PT":"PT",
              "Injured_S3":"PT"}
query.obs['celltype_merged2']=query.obs['celltype2'].map(celltypedict)

In [ ]:
celltypedict={"PT.1":"PT_Healthy","PT.2":"PT_Healthy","PT.3":"PT_Healthy","PT.4":"PT_Injury","PT.5":"PT_Injury","DTL_ATL":"DTL_ATL","TAL":"TAL",
              "DCT":"DCT","ICA":"ICA","ICB":"ICB","PC":"PC","Myeloid":"Myeloid","Lymphocyte":"Lymphocyte","PEC":"PEC","EC":"EC",
              "POD":"POD","Fib":"Fib","Pericyte":"Pericyte"}
train.obs['celltype_merged']=train.obs['nuctype2'].map(celltypedict)

In [ ]:
celltypedict={"PT":"PT","DTL_ATL":"DTL_ATL","TAL":"TAL",
              "DCT":"DCT","ICA":"ICA","ICB":"ICB","PC":"PC","Myeloid":"Myeloid","Lymphocyte":"Lymphocyte","PEC":"PEC","EC":"EC",
              "POD":"POD","Fib":"Fib","Pericyte":"Pericyte"}
train.obs['celltype_merged2']=train.obs['nuctype3'].map(celltypedict)

# Internal Validation

In [ ]:
train_pt=train[train.obs['celltype_merged2']=="PT",]
query_pt=query[query.obs['celltype_merged2']=="PT",]

In [ ]:
sc.pp.normalize_total(train_pt, target_sum=1e4)
sc.pp.log1p(train_pt)
sc.pp.highly_variable_genes(train_pt, min_mean=0.0125, max_mean=3, min_disp=0.5)

/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:155: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:01)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [ ]:
hvg=train_pt.var['highly_variable']
train_pt2=train_pt[:,hvg]

In [ ]:
train_pt2

View of AnnData object with n_obs × n_vars = 16011 × 2121
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'case_type', 'percent.mt', 'RNA_snn_res.0.6', 'seurat_clusters', 'nuctype1', 'nuctype2', 'nuctype3', 'celltype_merged', 'celltype_merged2'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [ ]:
np.random.seed(30729)
train_zoomin, val_zoomin = pySCN.splitCommonAnnData(train_pt, ncells=2000,dLevel="celltype_merged",cells_reserved=400)

PT_Injury : 
7247
PT_Healthy : 
8764


In [ ]:
train_zoomin2=train_zoomin[:,hvg]
val_zoomin2=val_zoomin[:,hvg]

In [ ]:
model_val=svm.SVC()
model_val.fit(train_zoomin2.X,train_zoomin2.obs.celltype_merged)

SVC()

In [ ]:
val_zoomin2.obs['predicted_type']=model_val.predict(val_zoomin2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(val_zoomin2.obs['predicted_type'],val_zoomin2.obs['celltype_merged'])

0.9548981240837315

In [ ]:
cd /content/drive/MyDrive/KPMP/snRNA/snRNA012022/svm/

/content/drive/MyDrive/KPMP/snRNA/snRNA012022/svm


In [ ]:
filename = 'svm_model_val_pt.sav'
pickle. dump(model_val, open(filename, 'wb'))

# Training with all human sample
# then query on mouse data

In [ ]:
model_all_pt=svm.SVC(cache_size=1000)
model_all_pt.fit(train_pt2.X,train_pt2.obs.celltype_merged)

SVC(cache_size=1000)

In [ ]:
query_pt2=query_pt[:,hvg]

In [ ]:
query_pt2.obs['predicted_type']=model_all_pt.predict(query_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(query_pt2.obs['predicted_type'],query_pt2.obs['celltype_merged'])

0.17426556243823954

In [ ]:
filename = 'svm_all_pt.sav'
pickle. dump(model_all_pt, open(filename, 'wb'))

# Now subgroup analysis

Control first

In [ ]:
train_control=train[train.obs['case_type']=="Healthy_Control",]
query_control=query[query.obs['Group']=="Control",]
train_control_pt=train_control[train_control.obs['celltype_merged2']=="PT",]
query_control_pt=query_control[query_control.obs['celltype_merged2']=="PT",]

In [ ]:
sc.pp.normalize_total(train_control_pt, target_sum=1e4)
sc.pp.log1p(train_control_pt)
sc.pp.highly_variable_genes(train_control_pt, min_mean=0.0125, max_mean=3, min_disp=0.5)
hvg=train_control_pt.var['highly_variable']
train_control_pt2=train_control_pt[:,hvg]

normalizing counts per cell
    finished (0:00:00)


/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:155: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [ ]:
model_control_pt=svm.SVC(cache_size=1000)
model_control_pt.fit(train_control_pt2.X,train_control_pt2.obs.celltype_merged)

SVC(cache_size=1000)

In [ ]:
query_control_pt2=query_control_pt[:,hvg]

In [ ]:
query_control_pt2.obs['predicted_type']=model_control_pt.predict(query_control_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(query_control_pt2.obs['predicted_type'],query_control_pt2.obs['celltype_merged'])

0.1355072831619405

In [ ]:
filename = 'svm_control_pt.sav'
pickle. dump(model_control_pt, open(filename, 'wb'))

Now AKI. query on 2days and 14days

In [ ]:
train_aki=train[train.obs['case_type']=="AKI",]
query_2d=query[query.obs['Group']=="2days",]
query_14d=query[query.obs['Group']=='14days',]
query_aki=query[query.obs['Group']!="Control",]

In [ ]:
train_aki_pt=train_aki[train_aki.obs['celltype_merged2']=="PT",]
query_2d_pt=query_2d[query_2d.obs['celltype_merged2']=="PT",]
query_14d_pt=query_14d[query_14d.obs['celltype_merged2']=="PT",]
query_aki_pt=query_aki[query_aki.obs['celltype_merged2']=="PT",]

In [ ]:
sc.pp.normalize_total(train_aki_pt, target_sum=1e4)
sc.pp.log1p(train_aki_pt)
sc.pp.highly_variable_genes(train_aki_pt, min_mean=0.0125, max_mean=3, min_disp=0.5)
hvg=train_aki_pt.var['highly_variable']
train_aki_pt2=train_aki_pt[:,hvg]

normalizing counts per cell
    finished (0:00:00)


/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:155: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [ ]:
model_aki_pt=svm.SVC(cache_size=1000)
model_aki_pt.fit(train_aki_pt2.X,train_aki_pt2.obs.celltype_merged)

SVC(cache_size=1000)

In [ ]:
query_2d_pt2=query_2d_pt[:,hvg]
query_14d_pt2=query_14d_pt[:,hvg]
query_aki_pt2=query_aki_pt[:,hvg]

In [ ]:
query_2d_pt2.obs['predicted_type']=model_aki_pt.predict(query_2d_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(query_2d_pt2.obs['predicted_type'],query_2d_pt2.obs['celltype_merged'])

0.0958228262782228

In [ ]:
query_14d_pt2.obs['predicted_type']=model_aki_pt.predict(query_14d_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(query_14d_pt2.obs['predicted_type'],query_14d_pt2.obs['celltype_merged'])

0.09401191331050218

In [ ]:
query_aki_pt2.obs['predicted_type']=model_aki_pt.predict(query_aki_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(query_aki_pt2.obs['predicted_type'],query_aki_pt2.obs['celltype_merged'])

0.14837826442238045

In [ ]:
filename = 'svm_aki_pt.sav'
pickle. dump(model_aki_pt, open(filename, 'wb'))

# Alternative Annotation

In [ ]:
celltypedict={"Healthy_S1S2":"PT_Healthy","Healthy_S3":"PT_Healthy","Fib":"Fib","EC1":"EC","Failed_Repair_PT":"PT_FR","DCT":"DCT","CTAL1":"TAL","CTAL2":"TAL",
              "DTL-ATL":"DTL-ATL","ICB":"ICB","CNT":"CNT","DCT-CNT":"DCT","ICA":"ICA","MTAL":"TAL","Mo":"Myeloid","PC1":"PC","Per":"Pericyte","PC2":"PC","Tcell":"Lymphocyte",
              "POD":"POD","PEC":"PEC","URO":"URO","MD":"MD","EC2":"EC","Repairing_PT":"PT_Injury","Injured_S1S2":"PT_Injury","Severe_Injured_PT":"PT_Injury",
              "Injured_S3":"PT_Injury"}
query.obs['celltype_merged']=query.obs['celltype2'].map(celltypedict)

In [ ]:
celltypedict={"PT.1":"PT_Healthy","PT.2":"PT_Healthy","PT.3":"PT_Healthy","PT.4":"PT_Injury","PT.5":"PT_FR","DTL_ATL":"DTL_ATL","TAL":"TAL",
              "DCT":"DCT","ICA":"ICA","ICB":"ICB","PC":"PC","Myeloid":"Myeloid","Lymphocyte":"Lymphocyte","PEC":"PEC","EC":"EC",
              "POD":"POD","Fib":"Fib","Pericyte":"Pericyte"}
train.obs['celltype_merged']=train.obs['nuctype2'].map(celltypedict)

In [ ]:
train_pt=train[train.obs['celltype_merged2']=="PT",]
query_pt=query[query.obs['celltype_merged2']=="PT",]

In [ ]:
sc.pp.normalize_total(train_pt, target_sum=1e4)
sc.pp.log1p(train_pt)
sc.pp.highly_variable_genes(train_pt, min_mean=0.0125, max_mean=3, min_disp=0.5)

/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:155: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [ ]:
hvg=train_pt.var['highly_variable']
train_pt2=train_pt[:,hvg]

In [ ]:
np.random.seed(30729)
train_zoomin, val_zoomin = pySCN.splitCommonAnnData(train_pt, ncells=2000,dLevel="celltype_merged",cells_reserved=400)

PT_Injury : 
2555
PT_Healthy : 
8764
PT_FR : 
4692


In [ ]:
train_zoomin2=train_zoomin[:,hvg]
val_zoomin2=val_zoomin[:,hvg]

In [ ]:
model_val=svm.SVC()
model_val.fit(train_zoomin2.X,train_zoomin2.obs.celltype_merged)

SVC()

In [ ]:
val_zoomin2.obs['predicted_type']=model_val.predict(val_zoomin2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(val_zoomin2.obs['predicted_type'],val_zoomin2.obs['celltype_merged'])

0.9036545981595688

In [ ]:
filename = 'svm_model_val_pt2.sav'
pickle. dump(model_val, open(filename, 'wb'))

In [ ]:
train_pt2=train_pt[:,hvg]
query_pt2=query_pt[:,hvg]

In [ ]:
model_all_pt=svm.SVC(cache_size=1000)
model_all_pt.fit(train_pt2.X,train_pt2.obs.celltype_merged)

SVC(cache_size=1000)

In [ ]:
query_pt2.obs['predicted_type']=model_all_pt.predict(query_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [ ]:
cohen_kappa_score(query_pt2.obs['predicted_type'],query_pt2.obs['celltype_merged'])

0.06588802049306453

In [ ]:
filename = 'svm_all_pt2.sav'
pickle. dump(model_all_pt, open(filename, 'wb'))

Control first

In [65]:
train_control=train[train.obs['case_type']=="Healthy_Control",]
query_control=query[query.obs['Group']=="Control",]
train_control_pt=train_control[train_control.obs['celltype_merged2']=="PT",]
query_control_pt=query_control[query_control.obs['celltype_merged2']=="PT",]

In [66]:
sc.pp.normalize_total(train_control_pt, target_sum=1e4)
sc.pp.log1p(train_control_pt)
sc.pp.highly_variable_genes(train_control_pt, min_mean=0.0125, max_mean=3, min_disp=0.5)

/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:155: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [67]:
hvg=train_control_pt.var['highly_variable']
train_control_pt2=train_control_pt[:,hvg]

In [68]:
model_control_pt=svm.SVC(cache_size=1000)
model_control_pt.fit(train_control_pt2.X,train_control_pt2.obs.celltype_merged)

SVC(cache_size=1000)

In [69]:
query_control_pt2=query_control_pt[:,hvg]

In [70]:
query_control_pt2.obs['predicted_type']=model_control_pt.predict(query_control_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [71]:
cohen_kappa_score(query_control_pt2.obs['predicted_type'],query_control_pt2.obs['celltype_merged'])

0.08541918223267464

In [72]:
filename = 'svm_control_pt2.sav'
pickle. dump(model_control_pt, open(filename, 'wb'))

Now AKI. query on 2days and 14days

In [73]:
train_aki=train[train.obs['case_type']=="AKI",]
query_2d=query[query.obs['Group']=="2days",]
query_14d=query[query.obs['Group']=='14days',]
query_aki=query[query.obs['Group']!="Control",]

In [74]:
train_aki_pt=train_aki[train_aki.obs['celltype_merged2']=="PT",]
query_2d_pt=query_2d[query_2d.obs['celltype_merged2']=="PT",]
query_14d_pt=query_14d[query_14d.obs['celltype_merged2']=="PT",]
query_aki_pt=query_aki[query_aki.obs['celltype_merged2']=="PT",]

In [75]:
sc.pp.normalize_total(train_aki_pt, target_sum=1e4)
sc.pp.log1p(train_aki_pt)
sc.pp.highly_variable_genes(train_aki_pt, min_mean=0.0125, max_mean=3, min_disp=0.5)

normalizing counts per cell


/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:155: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [76]:
hvg=train_aki_pt.var['highly_variable']
train_aki_pt2=train_aki_pt[:,hvg]

In [77]:
model_aki_pt=svm.SVC(cache_size=1000)
model_aki_pt.fit(train_aki_pt2.X,train_aki_pt2.obs.celltype_merged)

SVC(cache_size=1000)

In [78]:
query_2d_pt2=query_2d_pt[:,hvg]
query_14d_pt2=query_14d_pt[:,hvg]
query_aki_pt2=query_aki_pt[:,hvg]

In [79]:
query_2d_pt2.obs['predicted_type']=model_aki_pt.predict(query_2d_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [80]:
cohen_kappa_score(query_2d_pt2.obs['predicted_type'],query_2d_pt2.obs['celltype_merged'])

0.02214758227999858

In [81]:
query_14d_pt2.obs['predicted_type']=model_aki_pt.predict(query_14d_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [82]:
cohen_kappa_score(query_14d_pt2.obs['predicted_type'],query_14d_pt2.obs['celltype_merged'])

0.045269917345107324

In [83]:
query_aki_pt2.obs['predicted_type']=model_aki_pt.predict(query_aki_pt2.X)

Trying to set attribute `.obs` of view, copying.


In [84]:
cohen_kappa_score(query_aki_pt2.obs['predicted_type'],query_aki_pt2.obs['celltype_merged'])

0.031078434452498027

In [85]:
filename = 'svm_aki_pt2.sav'
pickle. dump(model_aki_pt, open(filename, 'wb'))